# Preprocesamiento de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Conectar al Drive para acceder a los datos

In [ ]:

try:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE = "/content/drive/MyDrive"
except Exception as e:
    print("No estás en Colab o ya está montado. Usa ruta local.")
    BASE = "."


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, glob
import pandas as pd
import unicodedata

# Carpeta donde están los Excel
CARPETA = os.path.join(BASE, "DATA_LIMPIEZA")

# Busca sólo .xlsx (si tuvieras .xls cambia el patrón o agrega otro glob)
rutas = sorted(glob.glob(os.path.join(CARPETA, "*.xlsx")))

print(f"Encontrados {len(rutas)} archivos en: {CARPETA}")
for r in rutas[:3]:
    print("•", os.path.basename(r))


Encontrados 24 archivos en: /content/drive/MyDrive/DATA_LIMPIEZA
• CuidadCapital.xlsx
• DATA_COMPLETAcsv.xlsx
• Elprogreso.xlsx


## Normalizar los nombre de archivos

In [ ]:
def normaliza(texto: str) -> str:
    if texto is None:
        return ""
    # quita acentos
    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join(ch for ch in texto if not unicodedata.combining(ch))
    # limpieza básica
    texto = texto.strip()
    texto = texto.replace("\n", " ").replace("\r", " ")
    # espacios -> guiones bajos, minúsculas
    texto = "_".join(texto.split())
    return texto.lower()


## Mapea cada uno de ellos y muestra sus encabezados, Si hay una columna vacia la elimina

In [ ]:
encabezados_por_archivo = []  # lista de dicts para DataFrame resumen

for ruta in rutas:
    nombre = os.path.basename(ruta)
    try:
        # Lee solo encabezados (nrows=0) de la PRIMERA hoja
        df0 = pd.read_excel(ruta, sheet_name=0, nrows=0, engine="openpyxl")

        # Detectar columnas Unnamed
        cols_original = list(map(str, df0.columns.tolist()))
        cols_unnamed = [c for c in cols_original if c.startswith("Unnamed")]

        if cols_unnamed:
            print(f"\n⚠ {nombre} contiene columnas vacías: {cols_unnamed}")
            # Eliminar columnas Unnamed
            df0 = df0.loc[:, ~df0.columns.str.contains('^Unnamed')]
            # Actualizar lista de columnas originales sin las vacías
            cols_original = list(map(str, df0.columns.tolist()))

        # Normalizar nombres
        cols_normal = [normaliza(c) for c in cols_original]

        print(f"\n===== {nombre} =====")
        print("Encabezados (limpios):")
        print(cols_original)

        encabezados_por_archivo.append({
            "archivo": nombre,
            "num_columnas": len(cols_original),
            "encabezados_original": cols_original,
            "encabezados_normalizado": cols_normal,
            "set_normalizado": tuple(sorted(set(cols_normal))),
        })

    except Exception as e:
        print(f"\n===== {nombre} =====")
        print("ERROR al leer:", e)



⚠ CuidadCapital.xlsx contiene columnas vacías: ['Unnamed: 1']

===== CuidadCapital.xlsx =====
Encabezados (limpios):
['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

===== DATA_COMPLETAcsv.xlsx =====
Encabezados (limpios):
['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

⚠ Elprogreso.xlsx contiene columnas vacías: ['Unnamed: 1']

===== Elprogreso.xlsx =====
Encabezados (limpios):
['CODIGO', 'DISTRITO', 'DEPARTAMENTO', 'MUNICIPIO', 'ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO', 'SUPERVISOR', 'DIRECTOR', 'NIVEL', 'SECTOR', 'AREA', 'STATUS', 'MODALIDAD', 'JORNADA', 'PLAN', 'DEPARTAMENTAL']

⚠ Guatemala.xlsx contiene columnas vacías: ['Unnamed: 1']

===== Guatemala.xlsx =====
E

## Compara si cada archivo tiene las misma columnas

In [ ]:
res = pd.DataFrame(encabezados_por_archivo)

# Encuentra el "esquema modal" (el conjunto de columnas normalizadas más frecuente)
conteo_sets = res["set_normalizado"].value_counts()
esquema_modal = conteo_sets.index[0] if not conteo_sets.empty else tuple()
print("\n\n=== Resumen de esquemas ===")
print(conteo_sets)

# Marca si cada archivo coincide con el esquema modal
res["coincide_modal"] = res["set_normalizado"].apply(lambda s: s == esquema_modal)

# Para ver rápidamente quiénes NO coinciden
no_match = res[~res["coincide_modal"]].copy()

print("\nArchivos que NO coinciden con el esquema modal:")
if no_match.empty:
    print("✅ Todos coinciden con el mismo conjunto de encabezados (normalizados).")
else:
    for _, row in no_match.iterrows():
        base = set(esquema_modal)
        actual = set(row["set_normalizado"])
        faltantes = sorted(list(base - actual))
        extras = sorted(list(actual - base))
        print(f"\n• {row['archivo']}")
        print("  - Faltan (vs modal):", faltantes if faltantes else "—")
        print("  - Extras (vs modal):", extras if extras else "—")




=== Resumen de esquemas ===
set_normalizado
(area, codigo, departamental, departamento, direccion, director, distrito, establecimiento, jornada, modalidad, municipio, nivel, plan, sector, status, supervisor, telefono)    24
Name: count, dtype: int64

Archivos que NO coinciden con el esquema modal:
✅ Todos coinciden con el mismo conjunto de encabezados (normalizados).


## Unir todo los archivos en un CSV

In [ ]:
import os, glob, unicodedata
import pandas as pd
CARPETA = os.path.join(BASE, "DATA_LIMPIEZA")
PATRON = "*.xlsx"



rutas = []
for patron in ([PATRON] if isinstance(PATRON, str) else PATRON):
    rutas.extend(glob.glob(os.path.join(CARPETA, patron)))
rutas = sorted(rutas)

print(f"Archivos encontrados: {len(rutas)}")
for r in rutas[:5]: print("•", os.path.basename(r))


meta = []
for ruta in rutas:
    nombre = os.path.basename(ruta)
    try:
        df0 = pd.read_excel(ruta, sheet_name=0, nrows=0, engine="openpyxl")
        if df0.columns.str.contains('^Unnamed').any():
            print(f"⚠ {nombre}: eliminando columnas Unnamed en encabezados")
            df0 = df0.loc[:, ~df0.columns.str.contains('^Unnamed')]
        cols = list(map(str, df0.columns.tolist()))
        cols_norm = tuple(sorted(set(normaliza(c) for c in cols)))
        meta.append({"archivo": nombre, "ruta": ruta, "cols_original": cols, "cols_norm_set": cols_norm})
    except Exception as e:
        print(f"❌ {nombre}: error leyendo encabezados -> {e}")

meta_df = pd.DataFrame(meta)
if meta_df.empty:
    raise SystemExit("No se pudieron leer encabezados de ningún archivo.")


conteo = meta_df["cols_norm_set"].value_counts()
esquema_modal = conteo.index[0]
print("\n=== Esquema modal (set normalizado) ===")
print(esquema_modal)
print("\nFrecuencias de esquemas:")
print(conteo)

modal_ruta = meta_df[meta_df["cols_norm_set"] == esquema_modal]["ruta"].iloc[0]
df_modal = pd.read_excel(modal_ruta, engine="openpyxl")
df_modal = df_modal.loc[:, ~df_modal.columns.str.contains('^Unnamed')]
orden_canonico = [c for c in df_modal.columns]  # orden y nombres "bonitos" de referencia
canonico_norm = [normaliza(c) for c in orden_canonico]


acumulados = []
resumen = []
for _, row in meta_df.iterrows():
    ruta = row["ruta"]
    nombre = row["archivo"]
    try:
        df = pd.read_excel(ruta, engine="openpyxl")
        # quitar columnas Unnamed
        if df.columns.str.contains('^Unnamed').any():
            print(f"⚠ {nombre}: quitando columnas Unnamed")
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

        # normalizar nombres para mapear al canónico
        col_norm_actual = [normaliza(c) for c in df.columns]

        # construir un mapping: columna_actual -> nombre_canonico (por el norm)
        mapping = {}
        for c_act, c_norm in zip(df.columns, col_norm_actual):
            if c_norm in canonico_norm:
                # usar el nombre canónico correspondiente
                idx = canonico_norm.index(c_norm)
                mapping[c_act] = orden_canonico[idx]
            else:
                # columna no está en el esquema modal; la omitimos (o podrías conservarla)
                mapping[c_act] = None

        # renombrar y filtrar solo columnas canónicas
        df = df.rename(columns={k:v for k,v in mapping.items() if v is not None})
        df = df.loc[:, [c for c in orden_canonico if c in df.columns]]



        acumulados.append(df)
        resumen.append({"archivo": nombre, "filas": len(df), "columnas": len(df.columns)})

    except Exception as e:
        print(f"❌ {nombre}: error al cargar/alinear -> {e}")

if not acumulados:
    raise SystemExit("No se logró acumular ningún DataFrame.")

full = pd.concat(acumulados, ignore_index=True)

print("\n=== Resumen de carga ===")
print(pd.DataFrame(resumen))

print("\nShape final concatenado:", full.shape)
print("Columnas finales:", list(full.columns))
# === Guardar salidas ===
os.makedirs(CARPETA, exist_ok=True)

csv_out  = os.path.join(CARPETA, "DATA_COMPLETA.csv")
xlsx_out = os.path.join(CARPETA, "DATA_COMPLETA.xlsx")

# CSV (UTF-8 con BOM para abrir bien en Excel)
full.to_csv(csv_out, index=False, encoding="utf-8-sig")
print("✔ CSV guardado en:", csv_out)

# XLSX (una hoja llamada 'UNIDOS')
with pd.ExcelWriter(xlsx_out, engine="openpyxl") as w:
    full.to_excel(w, index=False, sheet_name="UNIDOS")
print("✔ XLSX guardado en:", xlsx_out)



Archivos encontrados: 24
• CuidadCapital.xlsx
• DATA_COMPLETAcsv.xlsx
• Elprogreso.xlsx
• Guatemala.xlsx
• Izabal.xlsx
⚠ CuidadCapital.xlsx: eliminando columnas Unnamed en encabezados
⚠ Elprogreso.xlsx: eliminando columnas Unnamed en encabezados
⚠ Guatemala.xlsx: eliminando columnas Unnamed en encabezados
⚠ Izabal.xlsx: eliminando columnas Unnamed en encabezados
⚠ Quetzaltenango.xlsx: eliminando columnas Unnamed en encabezados
⚠ Quiche.xlsx: eliminando columnas Unnamed en encabezados
⚠ Sanmarcos.xlsx: eliminando columnas Unnamed en encabezados
⚠ Santarosa.xlsx: eliminando columnas Unnamed en encabezados
⚠ altaverapaz.xlsx: eliminando columnas Unnamed en encabezados
⚠ bajaverapaz.xlsx: eliminando columnas Unnamed en encabezados
⚠ chimaltenango.xlsx: eliminando columnas Unnamed en encabezados
⚠ chiquimula.xlsx: eliminando columnas Unnamed en encabezados
⚠ escuintla.xlsx: eliminando columnas Unnamed en encabezados
⚠ huehuetenango.xlsx: eliminando columnas Unnamed en encabezados
⚠ jalapa.x

# Análisis del estado de los datos crudos (antes de limpiar)

En esta sección se caracteriza el estado original de los datos sin aplicar transformaciones de limpieza:
- i) inventario y esquema por archivo
- ii) valores faltantes y tipos, iii) duplicados
- iv) problemas típicos en **ESTABLECIMIENTO**, **DIRECCION** y **TELEFONO** (formato/consistencia),

- v) distribución de categorías clave.

se analiza el estado original del dataset `DATA_COMPLETA.csv` antes de realizar cualquier limpieza.
Incluye: revisión de encabezados, valores faltantes, tipos de datos, duplicados y calidad de variables críticas (`ESTABLECIMIENTO`, `DIRECCION`, `TELEFONO`).



In [ ]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd

OUT_DIR = os.path.join(BASE, "DATA_LIMPIEZA", "evidencias_crudo")
os.makedirs(OUT_DIR, exist_ok=True)


PATH_CSV_CRUDO = os.path.join(BASE, "DATA_LIMPIEZA", "DATA_COMPLETA.csv")


try:
    df_raw = pd.read_csv(PATH_CSV_CRUDO, dtype=str, keep_default_na=False, na_values=[""])
except Exception as e:
    df_raw = pd.read_csv("DATA_COMPLETA.csv", dtype=str, keep_default_na=False, na_values=[""])

print(df_raw.shape)
df_raw.head(3)


(33831, 17)


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,00-01-0158-46,01-312,CIUDAD CAPITAL,ZONA 1,COLEGIO TECNICO PROGRESISTA CETECPRO,2A. AVENIDA 3-59,22512759,AMADO SALOMON FLORES PEREZ,IRMA AMPARO TOLEDO HERNANDEZ,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),GUATEMALA NORTE
1,00-01-0160-46,NaN,CIUDAD CAPITAL,ZONA 1,INSTITUTO DE EDUCACION DIVERSIFICADA 'CENTRO D...,8A AVE. 3-50,22329819,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
2,00-01-0162-46,01-102,CIUDAD CAPITAL,ZONA 1,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFI...,11 CALLE 9-33,22381718,ZIZI ARELY LOPEZ CHINCHILLA,----,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE


## Valores faltantes por columna (conteo y %)

In [ ]:
# Foto de columnas crudo (orden y nombres exactos)
cols_crudo = pd.DataFrame({"columna": df_raw.columns})
cols_crudo.to_csv(os.path.join(OUT_DIR, "01_cols_crudo.csv"), index=False)

# Tipos "inferidos" suavemente (para diagnóstico, no para limpieza)
tipos = df_raw.infer_objects(copy=False).dtypes.astype(str).reset_index()
tipos.columns = ["columna", "tipo_inferido"]
tipos.to_csv(os.path.join(OUT_DIR, "02_tipos_inferidos.csv"), index=False)

print("Columnas crudo:")
display(cols_crudo)
print("\nTipos inferidos:")
display(tipos)

print("\nMuestra cruda (5 filas):")
display(df_raw.head(5))


Columnas crudo:


,columna
0,CODIGO
1,DISTRITO
2,DEPARTAMENTO
3,MUNICIPIO
4,ESTABLECIMIENTO
5,DIRECCION
6,TELEFONO
7,SUPERVISOR
8,DIRECTOR
9,NIVEL



Tipos inferidos:


,columna,tipo_inferido
0,CODIGO,object
1,DISTRITO,object
2,DEPARTAMENTO,object
3,MUNICIPIO,object
4,ESTABLECIMIENTO,object
5,DIRECCION,object
6,TELEFONO,object
7,SUPERVISOR,object
8,DIRECTOR,object
9,NIVEL,object



Muestra cruda (5 filas):


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,00-01-0158-46,01-312,CIUDAD CAPITAL,ZONA 1,COLEGIO TECNICO PROGRESISTA CETECPRO,2A. AVENIDA 3-59,22512759,AMADO SALOMON FLORES PEREZ,IRMA AMPARO TOLEDO HERNANDEZ,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),GUATEMALA NORTE
1,00-01-0160-46,NaN,CIUDAD CAPITAL,ZONA 1,INSTITUTO DE EDUCACION DIVERSIFICADA 'CENTRO D...,8A AVE. 3-50,22329819,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
2,00-01-0162-46,01-102,CIUDAD CAPITAL,ZONA 1,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFI...,11 CALLE 9-33,22381718,ZIZI ARELY LOPEZ CHINCHILLA,----,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
3,00-01-0168-46,NaN,CIUDAD CAPITAL,ZONA 1,INSTITUTO DE EDUCACION DIVERSIFICADA 'LICEO MA...,2A AVE. 9-82,NaN,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),GUATEMALA NORTE
4,00-01-0173-46,01-301,CIUDAD CAPITAL,ZONA 1,INSTITUTO NORMAL PARA SEÑORITAS CENTRO AMERICA,1A CALLE 2-64,22323424,LUCRECIA MARISOL CERMEÑO GONZÁLEZ,INGRID MARIELA ESPINA ORELLANA,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE


In [ ]:
faltantes = df_raw.replace({"": np.nan}).isna().sum().to_frame("faltantes")
faltantes["total"] = len(df_raw)
faltantes["porcentaje"] = (faltantes["faltantes"] / faltantes["total"] * 100).round(2)
faltantes = faltantes.sort_values("porcentaje", ascending=False).reset_index().rename(columns={"index": "columna"})
faltantes.to_csv(os.path.join(OUT_DIR, "03_faltantes_por_columna.csv"), index=False)
display(faltantes)


,columna,faltantes,total,porcentaje
0,DIRECTOR,4497,33831,13.29
1,TELEFONO,2805,33831,8.29
2,SUPERVISOR,1584,33831,4.68
3,DISTRITO,1575,33831,4.66
4,DIRECCION,231,33831,0.68
5,ESTABLECIMIENTO,12,33831,0.04
6,CODIGO,0,33831,0.00
7,MUNICIPIO,0,33831,0.00
8,DEPARTAMENTO,0,33831,0.00
9,NIVEL,0,33831,0.00


## Duplicados (filas completas) y duplicados por clave lógica

Duplicado completo: misma fila exacta en todas las columnas.


In [ ]:
# Duplicados completos
dups_full_count = df_raw.duplicated(keep=False).sum()
print(f"Duplicados completos (todas las columnas): {dups_full_count}")

dups_full_preview = df_raw[df_raw.duplicated(keep=False)].head(50)
dups_full_preview.to_csv(os.path.join(OUT_DIR, "04_duplicados_completos_preview.csv"), index=False)

# Duplicados por clave lógica
CLAVE = ["ESTABLECIMIENTO", "DIRECCION", "TELEFONO"]
claves_presentes = [c for c in CLAVE if c in df_raw.columns]
print("Clave lógica utilizada:", claves_presentes)

if claves_presentes:
    dups_key = df_raw[df_raw.duplicated(subset=claves_presentes, keep=False)]
    dups_key.to_csv(os.path.join(OUT_DIR, "05_duplicados_por_clave.csv"), index=False)
    print(f"Duplicados por clave ({claves_presentes}): {len(dups_key)}")
    display(dups_key.head(20))
else:
    print("⚠️ No se encontraron todas las columnas de la clave sugerida para revisar duplicados por clave.")


Duplicados completos (todas las columnas): 33122
Clave lógica utilizada: ['ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO']
Duplicados por clave (['ESTABLECIMIENTO', 'DIRECCION', 'TELEFONO']): 33298


,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,00-01-0158-46,01-312,CIUDAD CAPITAL,ZONA 1,COLEGIO TECNICO PROGRESISTA CETECPRO,2A. AVENIDA 3-59,22512759,AMADO SALOMON FLORES PEREZ,IRMA AMPARO TOLEDO HERNANDEZ,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,DOBLE,DIARIO(REGULAR),GUATEMALA NORTE
1,00-01-0160-46,NaN,CIUDAD CAPITAL,ZONA 1,INSTITUTO DE EDUCACION DIVERSIFICADA 'CENTRO D...,8A AVE. 3-50,22329819,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
2,00-01-0162-46,01-102,CIUDAD CAPITAL,ZONA 1,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFI...,11 CALLE 9-33,22381718,ZIZI ARELY LOPEZ CHINCHILLA,----,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
3,00-01-0168-46,NaN,CIUDAD CAPITAL,ZONA 1,INSTITUTO DE EDUCACION DIVERSIFICADA 'LICEO MA...,2A AVE. 9-82,NaN,NaN,NaN,DIVERSIFICADO,PRIVADO,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),GUATEMALA NORTE
4,00-01-0173-46,01-301,CIUDAD CAPITAL,ZONA 1,INSTITUTO NORMAL PARA SEÑORITAS CENTRO AMERICA,1A CALLE 2-64,22323424,LUCRECIA MARISOL CERMEÑO GONZÁLEZ,INGRID MARIELA ESPINA ORELLANA,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
5,00-01-0174-46,01-307,CIUDAD CAPITAL,ZONA 1,ESCUELA NACIONAL CENTRAL DE FORMACION SECRETARIAL,12 AVENIDA 9-27,22322985,NORMA ARACELY PALOMO FRANCO DE DIAZ,FRANCISCO RENÉ CONTRERAS AQUINO,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
6,00-01-0175-46,01-317,CIUDAD CAPITAL,ZONA 1,ESCUELA NORMAL PARA MAESTROS DE EDUCACION MUSI...,2A. CALLE 4-04,22516651,MARIANO DOMINGO ESTRADA TELETOR,ALEIDA EMPERATRIZ PIÑON BONILLA,DIVERSIFICADO,OFICIAL,URBANA,CERRADA TEMPORALMENTE,MONOLINGUE,MATUTINA,DIARIO(REGULAR),GUATEMALA NORTE
7,00-01-0176-46,01-301,CIUDAD CAPITAL,ZONA 1,INSTITUTO NACIONAL DE BACHILLERATO EN COMPUTACION,3A CALLE 15-45,22202223,LUCRECIA MARISOL CERMEÑO GONZÁLEZ,FREDY HUMBERTO GONZÁLEZ SANTISTEBAN,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO(REGULAR),GUATEMALA NORTE
8,00-01-0177-46,01-318,CIUDAD CAPITAL,ZONA 1,ESCUELA NORMAL DE MAESTROS DE EDUCACION FISICA,22 CALLE 11-50 ZONA 1,24360852,LESLIE AZUCENA MONZON TECUN,------------,DIVERSIFICADO,OFICIAL,URBANA,CERRADA DEFINITIVAMENTE,MONOLINGUE,DOBLE,DIARIO(REGULAR),GUATEMALA NORTE
9,00-01-0178-46,01-403,CIUDAD CAPITAL,ZONA 1,ESCUELA NACIONAL CENTRAL DE CIENCIAS COMERCIALES,10A. AVENIDA 9-42,22320914,CARLOS HUMBERTO GONZÁLEZ DE LEÓN,ESWIN NOE ROSALES LÓPEZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO(REGULAR),GUATEMALA NORTE


## Diagnóstico de espacios y mayúsculas/minúsculas en campos críticos

Ayuda a justificar limpieza de espacios extra, saltos de línea y uniformidad de casing.

In [ ]:
def tiene_espacios_extras(s: pd.Series) -> pd.Series:
    # True si difiere al hacer strip o colapsar múltiples espacios
    return (s != s.str.strip()) | (s.str.replace(r"\s+", " ", regex=True) != s)

def ratio_mayusculas(s: pd.Series) -> float:
    s2 = s.fillna("")
    total = (s2.str.len()).sum()
    if total == 0: return 0.0
    mayus = s2.apply(lambda x: sum(ch.isupper() for ch in x)).sum()
    return round(mayus / total, 4)

diag_espacios = []
for col in ["ESTABLECIMIENTO","DIRECCION"]:
    if col in df_raw.columns:
        serie = df_raw[col].astype(str)
        extra = tiene_espacios_extras(serie).mean() * 100
        diag_espacios.append({"columna": col, "%_con_espacios_extras": round(extra, 2),
                              "ratio_mayusculas_en_texto": ratio_mayusculas(serie)})

diag_espacios_df = pd.DataFrame(diag_espacios)
diag_espacios_df.to_csv(os.path.join(OUT_DIR, "06_diag_espacios_casing.csv"), index=False)
display(diag_espacios_df)


,columna,%_con_espacios_extras,ratio_mayusculas_en_texto
0,ESTABLECIMIENTO,0.0,0.8743
1,DIRECCION,0.0,0.6785


## Teléfonos crudos: caracteres no numéricos y longitud ≠ 8

Regla base para Guatemala: 8 dígitos. Guardamos ejemplos problemáticos para el informe.

In [ ]:
if "TELEFONO" in df_raw.columns:
    tel = df_raw["TELEFONO"].astype(str)

    # Tel no vacíos
    tel_no_vacios = tel.replace({"": np.nan}).dropna()

    # Con caracteres no numéricos
    tel_no_numericos_mask = tel_no_vacios.str.contains(r"[^0-9]", regex=True)
    tel_no_numericos = df_raw.loc[tel_no_vacios.index[tel_no_numericos_mask]]
    tel_no_numericos.to_csv(os.path.join(OUT_DIR, "07_telefonos_no_numericos.csv"), index=False)

    # Longitud distinta a 8 (después de quitar no numéricos por diagnóstico)
    tel_norm = tel_no_vacios.str.replace(r"[^0-9]", "", regex=True)
    tel_len_distinta = df_raw.loc[tel_norm.index[(tel_norm.str.len() != 8)]]
    tel_len_distinta.to_csv(os.path.join(OUT_DIR, "08_telefonos_longitud_invalida.csv"), index=False)

    resumen_tel = pd.DataFrame({
        "total_no_vacios": [len(tel_no_vacios)],
        "con_no_numericos": [tel_no_numericos.shape[0]],
        "longitud_!=8": [tel_len_distinta.shape[0]]
    })
    resumen_tel.to_csv(os.path.join(OUT_DIR, "09_resumen_telefonos.csv"), index=False)
    display(resumen_tel)
else:
    print("⚠️ No existe columna TELEFONO en el CSV crudo.")


,total_no_vacios,con_no_numericos,longitud_!=8
0,33831,4129,4284


## Direcciones crudas: patrones y abreviaturas comunes

Checamos presencia de “Zona”, “Av.”, “Calz.”, “Km”, números de calle/avenida, etc.

In [ ]:
patrones = {
    "contiene_zona": r"\b(zona|zn)\b",
    "contiene_av": r"\b(av\.?|avenida)\b",
    "contiene_calle": r"\b(calle|cll\.)\b",
    "contiene_calzada": r"\b(calz\.?|calzada)\b",
    "contiene_km": r"\bkm\b",
    "tiene_numero": r"\d+"
}

if "DIRECCION" in df_raw.columns:
    dire = df_raw["DIRECCION"].astype(str).str.lower()
    resumen_dir = {}
    total_dir_no_vacias = dire.replace({"": np.nan}).dropna().shape[0]
    resumen_dir["total_no_vacias"] = total_dir_no_vacias

    for nombre, pat in patrones.items():
        resumen_dir[nombre] = int(dire.str.contains(pat, regex=True).sum())

    resumen_dir_df = pd.DataFrame([resumen_dir])
    resumen_dir_df.to_csv(os.path.join(OUT_DIR, "10_resumen_direcciones.csv"), index=False)
    display(resumen_dir_df.head())
else:
    print("⚠️ No existe columna DIRECCION en el CSV crudo.")


/tmp/ipython-input-4028851540.py:17: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  resumen_dir[nombre] = int(dire.str.contains(pat, regex=True).sum())


,total_no_vacias,contiene_zona,contiene_av,contiene_calle,contiene_calzada,contiene_km,tiene_numero
0,33831,15111,9255,10704,729,762,24297


## Posibles duplicados por variaciones tipográficas en ESTABLECIMIENTO

Usamos una clave “normalizada” (sin acentos, minúsculas, espacios colapsados) SOLO para diagnóstico.

In [ ]:
import unicodedata
import re

def normaliza_nombre(serie):
    """
    Normaliza texto en una Serie de pandas:
    - Quita acentos
    - Convierte a mayúsculas
    - Elimina caracteres no alfanuméricos (excepto espacios)
    - Elimina espacios múltiples
    """
    def limpiar(texto):
        if pd.isna(texto):
            return ""
        # Quitar acentos
        texto = ''.join(c for c in unicodedata.normalize('NFD', str(texto))
                        if unicodedata.category(c) != 'Mn')
        # Mayúsculas
        texto = texto.upper()
        # Dejar solo letras, números y espacios
        texto = re.sub(r'[^A-Z0-9 ]+', ' ', texto)
        # Quitar espacios extra
        texto = re.sub(r'\s+', ' ', texto).strip()
        return texto

    return serie.astype(str).map(limpiar)


In [ ]:


if "ESTABLECIMIENTO" in df_raw.columns:
    est_norm = normaliza_nombre(df_raw["ESTABLECIMIENTO"])
    df_tmp = df_raw.copy()
    df_tmp["_EST_NORM"] = est_norm

    # grupos con el mismo normalizado
    grp = df_tmp.groupby("_EST_NORM").size().reset_index(name="conteo").sort_values("conteo", ascending=False)
    grp = grp[grp["conteo"] > 1]  # solo los que aparecen repetidos
    grp.to_csv(os.path.join(OUT_DIR, "11_establecimientos_norm_repetidos.csv"), index=False)

    # Muestra de los top 30 grupos repetidos
    candidatos = df_tmp[df_tmp["_EST_NORM"].isin(grp["_EST_NORM"].head(30))]
    candidatos = candidatos.sort_values("_EST_NORM")
    candidatos.to_csv(os.path.join(OUT_DIR, "12_establecimientos_posibles_variaciones.csv"), index=False)

    print("Grupos con nombre normalizado repetido (top 10):")
    display(grp.head(10))
else:
    print("⚠️ No existe columna ESTABLECIMIENTO en el CSV crudo.")


Grupos con nombre normalizado repetido (top 10):


,_EST_NORM,conteo
3423,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,1398
67,CENTRO DE EDUCACION EXTRAESCOLAR CEEX,165
3043,INSTITUTO DE EDUCACION DIVERSIFICADA POR COOPE...,141
3158,INSTITUTO DIVERSIFICADO POR COOPERATIVA,81
2978,INSTITUTO DE COMPUTACION INFORMATICA,81
4988,PROGRAMA NACIONAL DE EDUCACION ALTERNATIVA PRONEA,81
14,ASOCIACION DE MAESTROS DE EDUCACION RURAL DE G...,78
2772,ESCUELA NORMAL DE EDUCACION FISICA,72
4846,LICEO SAN JOSE,69
3448,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA ...,66


## Resumen ejecutivo del estado crudo

In [ ]:
resumen_crudo = {
    "filas_totales": len(df_raw),
    "columnas_totales": len(df_raw.columns),
    "duplicados_completos": int(df_raw.duplicated(keep=False).sum()),
    "columnas_con_faltantes": int((df_raw.replace({"": np.nan}).isna().sum() > 0).sum())
}

# Porcentaje de filas con TEL no válido (si existe)
if "TELEFONO" in df_raw.columns:
    tel = df_raw["TELEFONO"].astype(str)
    tel_no_vacios = tel.replace({"": np.nan}).dropna()
    tel_norm = tel_no_vacios.str.replace(r"[^0-9]", "", regex=True)
    tel_invalidos = (tel_norm.str.len() != 8).sum()
    resumen_crudo["telefonos_invalidos"] = int(tel_invalidos)
    resumen_crudo["%_telefonos_invalidos_sobre_no_vacios"] = round(100 * tel_invalidos / max(1, len(tel_no_vacios)), 2)

# Columnas con mayor % de faltantes (top 5)
falt_top5 = (df_raw.replace({"": np.nan}).isna().mean() * 100).sort_values(ascending=False).head(5).round(2)
resumen_crudo_df = pd.DataFrame([resumen_crudo])
resumen_crudo_df.to_csv(os.path.join(OUT_DIR, "13_resumen_crudo.csv"), index=False)

print("Resumen crudo:")
display(resumen_crudo_df)
print("\nTop 5 columnas con mayor % de faltantes:")
display(falt_top5.to_frame("porcentaje_faltante"))


Resumen crudo:


,filas_totales,columnas_totales,duplicados_completos,columnas_con_faltantes,telefonos_invalidos,%_telefonos_invalidos_sobre_no_vacios
0,11277,17,0,6,1900,16.85



Top 5 columnas con mayor % de faltantes:


,porcentaje_faltante
DIRECTOR,13.29
TELEFONO,8.29
SUPERVISOR,4.68
DISTRITO,4.66
DIRECCION,0.68


In [ ]:
# ================================================================
# Análisis del estado de los datos crudos (SIN LIMPIAR)
# Para columnas:
# ['CODIGO','DISTRITO','DEPARTAMENTO','MUNICIPIO','ESTABLECIMIENTO',
#  'DIRECCION','TELEFONO','SUPERVISOR','DIRECTOR','NIVEL','SECTOR',
#  'AREA','STATUS','MODALIDAD','JORNADA','PLAN','DEPARTAMENTAL']
# ================================================================

import pandas as pd
import numpy as np
import re
import unicodedata

# ---------- Config ----------
CAMPOS = ['CODIGO','DISTRITO','DEPARTAMENTO','MUNICIPIO','ESTABLECIMIENTO',
          'DIRECCION','TELEFONO','SUPERVISOR','DIRECTOR','NIVEL','SECTOR',
          'AREA','STATUS','MODALIDAD','JORNADA','PLAN','DEPARTAMENTAL']

df = df_raw.copy()  # si tu df se llama diferente, cámbialo aquí

# Validación de columnas presentes
faltantes = [c for c in CAMPOS if c not in df.columns]
if faltantes:
    raise ValueError(f"Faltan columnas en el DataFrame: {faltantes}")

# ---------- Helpers de observación (no limpian, solo vista previa) ----------
def str_preview(x, k=5):
    vals = pd.Series(x).dropna().astype(str).unique()[:k]
    return "; ".join(map(str, vals))

def solo_digitos(s):
    if pd.isna(s):
        return ""
    return re.sub(r"\D", "", str(s))

def tiene_multiples_telefonos(s):
    # indicios de múltiple teléfono en una celda
    return bool(re.search(r"[;/,]| y | o ", str(s), flags=re.IGNORECASE))

def normaliza_suave(texto):
    """Normalización ligera SOLO para diagnóstico de posibles duplicados;
    no afecta el df original."""
    if pd.isna(texto):
        return ""
    t = str(texto)
    t = ''.join(c for c in unicodedata.normalize('NFD', t) if unicodedata.category(c) != 'Mn')
    t = t.upper()
    t = re.sub(r'[^A-Z0-9 ]+', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

# ---------- 1) Resumen general por columna ----------
rows = []
n = len(df)
for col in CAMPOS:
    s = df[col]
    n_null = s.isna().sum()
    n_empty = (s.astype(str).str.strip().eq("") | s.isna()).sum()
    nunique = s.nunique(dropna=True)
    ejemplos = str_preview(s, k=6)
    rows.append({
        "columna": col,
        "filas": n,
        "nulos": n_null,
        "% nulos": round(100*n_null/n, 2),
        "vacíos/''": int(n_empty),
        "% vacíos": round(100*n_empty/n, 2),
        "valores_únicos": nunique,
        "ejemplos_crudos": ejemplos
    })

reporte_general = pd.DataFrame(rows).sort_values(["% nulos","% vacíos","valores_únicos"], ascending=[False, False, True])
print("=== Resumen general (crudo) ===")
display(reporte_general)

# 2.1 CODIGO: unicidad, formato
if "CODIGO" in df.columns:
    s = df["CODIGO"].astype(str)
    cod_unicos = s.nunique(dropna=True)
    dup = s[s.duplicated(keep=False)]
    print("\n=== CODIGO ===")
    print(f"IDs únicos: {cod_unicos} de {n} filas")
    if not dup.empty:
        print(f"Posibles duplicados de CODIGO (muestran primeras 10 filas):")
        display(df.loc[s.duplicated(keep=False), ["CODIGO","ESTABLECIMIENTO","DEPARTAMENTO","MUNICIPIO"]].head(10))

# 2.2 DEPARTAMENTO y MUNICIPIO: cardinalidad y cruces básicos
print("\n=== DEPARTAMENTO / MUNICIPIO ===")
print("Departamentos únicos:", df["DEPARTAMENTO"].nunique())
print(sorted(df["DEPARTAMENTO"].dropna().unique())[:30])
print("\nEjemplos de MUNICIPIO por departamento (primeros 10 dptos):")
ejemplos_dm = (df.groupby("DEPARTAMENTO")["MUNICIPIO"]
                 .apply(lambda s: sorted(pd.Series(s.dropna().unique()).astype(str))[:8])
                 .reset_index(name="ejemplos_municipios"))
display(ejemplos_dm.head(10))

# 2.3 ESTABLECIMIENTO: posibles duplicados por normalización suave (solo diagnóstico)
print("\n=== ESTABLECIMIENTO (diagnóstico de posibles duplicados) ===")
est_norm = df["ESTABLECIMIENTO"].map(normaliza_suave)
grp_est = (pd.DataFrame({"EST": df["ESTABLECIMIENTO"], "_NORM": est_norm})
             .groupby("_NORM").agg(n=("EST","size"),
                                   ejemplos=("EST", lambda s: list(pd.Series(s).unique())[:3]))
             .reset_index()
             .sort_values("n", ascending=False))
display(grp_est.head(15))
print("Interpretación: grupos con n>1 podrían ser el mismo establecimiento escrito distinto (ver 'ejemplos').")

# 2.4 DIRECCION: patrones problemáticos comunes (solo conteos)
if "DIRECCION" in df.columns:
    s = df["DIRECCION"].astype(str)
    patrones = {
        "contiene 'ZONA O' (letra O)": s.str.contains(r"ZONA\s*O(?![0-9])", case=False, regex=True, na=False).sum(),
        "contiene abreviaturas AV/AV.": s.str.contains(r"\bAV\.?\b", case=False, regex=True, na=False).sum(),
        "contiene # o No.": s.str.contains(r"(#|NO\.)", case=False, regex=True, na=False).sum(),
        "caracteres no alfanum/espacio": s.str.contains(r"[^A-Za-z0-9ÁÉÍÓÚáéíóúÜüÑñ\s\.,#/\-]", regex=True, na=False).sum()
    }
    print("\n=== DIRECCION (patrones indicativos) ===")
    for k,v in patrones.items():
        print(f"{k}: {v} filas")
    print("\nMuestra de direcciones crudas:")
    display(df[["DIRECCION"]].sample(min(10, len(df)), random_state=1))

# 2.5 TELEFONO: longitud de dígitos, múltiples teléfonos, vacíos aparentes
if "TELEFONO" in df.columns:
    tel = df["TELEFONO"].astype(str)
    tel_digits = tel.map(solo_digitos)
    longitudes = tel_digits.str.len().value_counts().sort_index()
    multi = tel.map(tiene_multiples_telefonos).sum()
    vacios_aparentes = tel.str.strip().eq("").sum()
    print("\n=== TELEFONO (diagnóstico) ===")
    print("Distribución de longitudes (solo dígitos):")
    display(longitudes.to_frame("conteo").rename_axis("longitud_dígitos"))
    print(f"Filas con indicios de múltiples teléfonos en una celda: {multi}")
    print(f"Vací­os visibles ('' o espacios): {vacios_aparentes}")
    print("\nEjemplos crudos de teléfono:")
    display(df[["TELEFONO"]].sample(min(10, len(df)), random_state=2))

# 2.6 NIVEL / SECTOR / AREA / STATUS / MODALIDAD / JORNADA / PLAN / DISTRITO / DEPARTAMENTAL / SUPERVISOR / DIRECTOR
def top_valores(col, k=15):
    vc = df[col].value_counts(dropna=False).head(k)
    return vc.to_frame("conteo").rename_axis(col)

print("\n=== TOP valores por columna (hasta 15) ===")
for col in ['NIVEL','SECTOR','AREA','STATUS','MODALIDAD','JORNADA','PLAN',
            'DISTRITO','DEPARTAMENTAL','SUPERVISOR','DIRECTOR']:
    if col in df.columns:
        print(f"\n--- {col} ---")
        display(top_valores(col))

# 2.7 Consistencia básica: pares DEPARTAMENTO-MUNICIPIO muy raros (conteos bajos)
print("\n=== Pares DEPARTAMENTO-MUNICIPIO poco frecuentes (conteo <= 2) ===")
par_ct = (df.groupby(["DEPARTAMENTO","MUNICIPIO"]).size()
            .reset_index(name="conteo")
            .sort_values("conteo"))
display(par_ct[par_ct["conteo"] <= 2].head(20))


=== Resumen general (crudo) ===


,columna,filas,nulos,% nulos,vacíos/'',% vacíos,valores_únicos,ejemplos_crudos
8,DIRECTOR,33831,4497,13.29,4497,13.29,5249,IRMA AMPARO TOLEDO HERNANDEZ; ----; INGRID MAR...
6,TELEFONO,33831,2805,8.29,2805,8.29,6572,22512759; 22329819; 22381718; 22323424; 223229...
7,SUPERVISOR,33831,1584,4.68,1584,4.68,695,AMADO SALOMON FLORES PEREZ; ZIZI ARELY LOPEZ C...
1,DISTRITO,33831,1575,4.66,1575,4.66,738,01-312; 01-102; 01-301; 01-307; 01-317; 01-318
5,DIRECCION,33831,231,0.68,231,0.68,7068,2A. AVENIDA 3-59; 8A AVE. 3-50; 11 CALLE 9-33;...
4,ESTABLECIMIENTO,33831,12,0.04,12,0.04,6246,COLEGIO TECNICO PROGRESISTA CETECPRO; INSTITUT...
9,NIVEL,33831,0,0.00,0,0.00,1,DIVERSIFICADO
13,MODALIDAD,33831,0,0.00,0,0.00,2,MONOLINGUE; BILINGUE
11,AREA,33831,0,0.00,0,0.00,3,URBANA; RURAL; SIN ESPECIFICAR
10,SECTOR,33831,0,0.00,0,0.00,4,PRIVADO; OFICIAL; MUNICIPAL; COOPERATIVA



=== CODIGO ===
IDs únicos: 11277 de 33831 filas
Posibles duplicados de CODIGO (muestran primeras 10 filas):


,CODIGO,ESTABLECIMIENTO,DEPARTAMENTO,MUNICIPIO
0,00-01-0158-46,COLEGIO TECNICO PROGRESISTA CETECPRO,CIUDAD CAPITAL,ZONA 1
1,00-01-0160-46,INSTITUTO DE EDUCACION DIVERSIFICADA 'CENTRO D...,CIUDAD CAPITAL,ZONA 1
2,00-01-0162-46,INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFI...,CIUDAD CAPITAL,ZONA 1
3,00-01-0168-46,INSTITUTO DE EDUCACION DIVERSIFICADA 'LICEO MA...,CIUDAD CAPITAL,ZONA 1
4,00-01-0173-46,INSTITUTO NORMAL PARA SEÑORITAS CENTRO AMERICA,CIUDAD CAPITAL,ZONA 1
5,00-01-0174-46,ESCUELA NACIONAL CENTRAL DE FORMACION SECRETARIAL,CIUDAD CAPITAL,ZONA 1
6,00-01-0175-46,ESCUELA NORMAL PARA MAESTROS DE EDUCACION MUSI...,CIUDAD CAPITAL,ZONA 1
7,00-01-0176-46,INSTITUTO NACIONAL DE BACHILLERATO EN COMPUTACION,CIUDAD CAPITAL,ZONA 1
8,00-01-0177-46,ESCUELA NORMAL DE MAESTROS DE EDUCACION FISICA,CIUDAD CAPITAL,ZONA 1
9,00-01-0178-46,ESCUELA NACIONAL CENTRAL DE CIENCIAS COMERCIALES,CIUDAD CAPITAL,ZONA 1



=== DEPARTAMENTO / MUNICIPIO ===
Departamentos únicos: 23
['ALTA VERAPAZ', 'BAJA VERAPAZ', 'CHIMALTENANGO', 'CHIQUIMULA', 'CIUDAD CAPITAL', 'EL PROGRESO', 'ESCUINTLA', 'GUATEMALA', 'HUEHUETENANGO', 'IZABAL', 'JALAPA', 'JUTIAPA', 'PETEN', 'QUETZALTENANGO', 'QUICHE', 'RETALHULEU', 'SACATEPEQUEZ', 'SAN MARCOS', 'SANTA ROSA', 'SOLOLA', 'SUCHITEPEQUEZ', 'TOTONICAPAN', 'ZACAPA']

Ejemplos de MUNICIPIO por departamento (primeros 10 dptos):


...

,conteo
SUPERVISOR,
NaN,1584
CARLOS HUMBERTO GONZÁLEZ DE LEÓN,969
MIGUEL ANGEL ARMAS ROCHA,723
JUAN ENRIQUE MARTINEZ SOLANO,576
REMY ARTURO SINAY GUDIEL,528
ELENA ELIZABETH SUCHITE GARNICA DE QUINTANILLA,438
IDALIA DEL ROSARIO LOPEZ SANDOVAL DE PAIZ,369
MILTON ALONSO ALVAREZ FUENTES,354
JUAN FRANCISCO GODOY DAVILA,354



--- DIRECTOR ---


,conteo
DIRECTOR,
NaN,4497
---,501
--,123
----,120
THELMA ONDINA VASQUEZ,102
SIN DATO,84
-----,54
MARIA DEL ROSARIO LOPEZ ESCOBAR DE PAZ,42
-,42



=== Pares DEPARTAMENTO-MUNICIPIO poco frecuentes (conteo <= 2) ===


,DEPARTAMENTO,MUNICIPIO,conteo


In [ ]:
# ================================================================
# PARTE 4 — PIPELINE DE LIMPIEZA (REPRODUCIBLE)
# Limpia: ['CODIGO','DISTRITO','DEPARTAMENTO','MUNICIPIO','ESTABLECIMIENTO',
#          'DIRECCION','TELEFONO','SUPERVISOR','DIRECTOR','NIVEL','SECTOR',
#          'AREA','STATUS','MODALIDAD','JORNADA','PLAN','DEPARTAMENTAL']
# y exporta CSV + XLSX
# ================================================================

import os, re, unicodedata
import numpy as np
import pandas as pd

# -------- Entradas / Salidas --------
df = df_raw.copy()  # <--- si tu DF se llama distinto, cámbialo aquí
BASE = "/content/drive/MyDrive"  # ajusta si trabajas local
CARPETA = os.path.join(BASE, "DATA_LIMPIEZA")
os.makedirs(CARPETA, exist_ok=True)
CSV_OUT  = os.path.join(CARPETA, "DATA_COMPLETA_LIMPIA.csv")
XLSX_OUT = os.path.join(CARPETA, "DATA_COMPLETA_LIMPIA.xlsx")

CAMPOS = ['CODIGO','DISTRITO','DEPARTAMENTO','MUNICIPIO','ESTABLECIMIENTO',
          'DIRECCION','TELEFONO','SUPERVISOR','DIRECTOR','NIVEL','SECTOR',
          'AREA','STATUS','MODALIDAD','JORNADA','PLAN','DEPARTAMENTAL']

# Validación de columnas presentes
faltantes = [c for c in CAMPOS if c not in df.columns]
if faltantes:
    raise ValueError(f"Faltan columnas en el DataFrame: {faltantes}")

# ================================================================
# Helpers: normalización y utilidades
# ================================================================
def quitar_acentos(s: str) -> str:
    if pd.isna(s): return s
    s = str(s)
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    return s

def norm_texto(s: str, keep=" A-Z0-9", to_upper=True) -> str:
    """
    Normaliza texto: quita acentos, opcional mayúsculas, filtra caracteres.
    keep: clase de caracteres a permitir (regex dentro de [])
    """
    if pd.isna(s): return np.nan
    s = quitar_acentos(str(s))
    if to_upper: s = s.upper()
    s = re.sub(rf"[^{keep}]+", " ", s)  # deja solo permitidos
    s = re.sub(r"\s+", " ", s).strip()
    return s if s else np.nan

def norm_nombre_persona(s: str) -> str:
    # Para SUPERVISOR / DIRECTOR — conserva letras, espacios, apóstrofes básicos
    return norm_texto(s, keep="A-Z '", to_upper=True)

def norm_establecimiento(s: str) -> str:
    # Normalización robusta para detectar duplicados por variaciones tipográficas
    s = norm_texto(s, keep="A-Z0-9 ", to_upper=True)
    if pd.isna(s): return s
    # Expandir abreviaturas comunes (no destructivo, sólo homogeneizar)
    rep = {
        r"\bINST\b": "INSTITUTO",
        r"\bINST\.\b": "INSTITUTO",
        r"\bESC\b": "ESCUELA",
        r"\bESC\.\b": "ESCUELA",
        r"\bOF\b": "OFICIAL",
        r"\bMIX\b": "MIXTO",
        r"\bNAC\b": "NACIONAL",
        r"\bEDUC\b": "EDUCACION",
    }
    for pat, repl in rep.items():
        s = re.sub(pat, repl, s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def norm_direccion(s: str) -> str:
    # Homogeneiza direcciones (no “adivina” calles, sólo corrige lo recurrente)
    s = norm_texto(s, keep="A-Z0-9 #/.,-", to_upper=True)
    if pd.isna(s): return s
    rep = {
        r"\bAV\.?\b": "AVENIDA",
        r"\bCALZ\.?\b": "CALZADA",
        r"\bCll\b": "CALLE",
        r"\bNo\.\b": "NO",
        r"\bZONA\s*O\b": "ZONA 0",  # O (letra) -> 0 (cero)
    }
    for pat, repl in rep.items():
        s = re.sub(pat, repl, s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def solo_digitos(s: str) -> str:
    return re.sub(r"\D", "", str(s)) if not pd.isna(s) else ""

def norm_telefono_campo(s: str):
    """
    - separa por ; , / y ' y ' o ' o '
    - deja solo dígitos
    - mantiene sólo teléfonos de 8 dígitos (formato GT)
    - devuelve '; ' join o NaN si ninguno válido
    """
    if pd.isna(s) or str(s).strip() == "":
        return np.nan
    trozos = re.split(r"[;,/]| y | o ", str(s), flags=re.IGNORECASE)
    limpios = []
    for t in trozos:
        d = solo_digitos(t)
        if len(d) == 8:
            limpios.append(d)
    if not limpios:
        return np.nan
    # elimina duplicados preservando orden
    seen = set(); out = []
    for d in limpios:
        if d not in seen:
            seen.add(d); out.append(d)
    return "; ".join(out)

def map_categorias(s: pd.Series, mapping: dict, to_upper=True) -> pd.Series:
    """
    Normaliza a mayúsculas y aplica diccionario de equivalencias.
    Deja valores no mapeados tal cual (para no perder información).
    """
    x = s.astype(str).map(lambda v: norm_texto(v, keep="A-Z0-9 ", to_upper=to_upper))
    return x.map(lambda v: mapping.get(v, v) if pd.notna(v) else v)

# ================================================================
# Catálogos base (puedes ampliarlos según tus datos)
# ================================================================
DEPARTAMENTOS_OFICIALES = {
    "ALTA VERAPAZ","BAJA VERAPAZ","CHIMALTENANGO","CHIQUIMULA","EL PROGRESO","ESCUINTLA",
    "GUATEMALA","HUEHUETENANGO","IZABAL","JALAPA","JUTIAPA","PETEN","QUETZALTENANGO",
    "QUICHE","RETALHULEU","SACATEPEQUEZ","SAN MARCOS","SANTA ROSA","SOLOLA",
    "SUCHITEPEQUEZ","TOTONICAPAN","ZACAPA"
}

# equivalencias comunes (errores tipográficos) -> oficial
MAP_DPTO_EQUIV = {
    "QUICHE": "QUICHE",
    "QUICH E": "QUICHE",
    "SACATEPEQUEZ": "SACATEPEQUEZ",
    "SACATEPQUEZ": "SACATEPEQUEZ",
    "GUATEMALA": "GUATEMALA",
    "GUAUTEMALA": "GUATEMALA",
}

MAP_NIVEL = {
    "PREPRIMARIA":"PREPRIMARIA",
    "PRIMARIA":"PRIMARIA",
    "BASICO":"BASICO",
    "BASICO CICLO BASICO":"BASICO",
    "DIVERSIFICADO":"DIVERSIFICADO",
}

MAP_SECTOR = {
    "OFICIAL":"OFICIAL",
    "PRIVADO":"PRIVADO",
    "MIXTO":"PRIVADO",  # si en tus datos existe “MIXTO” como sector, decide política; aquí lo mapeo a PRIVADO
}

MAP_AREA = {"URBANA":"URBANA", "RURAL":"RURAL", "URB":"URBANA"}

MAP_STATUS = {"ACTIVO":"ACTIVO", "CERRADO":"CERRADO"}

MAP_JORNADA = {
    "MATUTINA":"MATUTINA","MAT":"MATUTINA",
    "VESPERTINA":"VESPERTINA","VESP":"VESPERTINA",
    "NOCTURNA":"NOCTURNA","NOCT":"NOCTURNA",
    "DOBLE":"DOBLE","JORNADA DOBLE":"DOBLE"
}

# ================================================================
# 1) Limpieza campo a campo (con justificación en comentarios)
# ================================================================
# CODIGO: mantener como texto (evita pérdida de ceros), strip
df["CODIGO"] = df["CODIGO"].astype(str).str.strip()

# DISTRITO: formato homogéneo (mayúsculas; ej. "DISTRITO 01")
df["DISTRITO"] = df["DISTRITO"].map(lambda s: norm_texto(s, keep="A-Z0-9 ", to_upper=True))

# DEPARTAMENTO: mayúsculas, sin acentos y mapeo a oficial si aplica
df["DEPARTAMENTO"] = df["DEPARTAMENTO"].map(lambda s: norm_texto(s, keep="A-Z ", to_upper=True))
df["DEPARTAMENTO"] = df["DEPARTAMENTO"].replace(MAP_DPTO_EQUIV)

# MUNICIPIO: normalización ligera (no corrige semántica de municipio–depto aquí)
df["MUNICIPIO"] = df["MUNICIPIO"].map(lambda s: norm_texto(s, keep="A-Z0-9 ", to_upper=True))

# ESTABLECIMIENTO: normalización robusta para duplicados tipográficos
df["ESTABLECIMIENTO"] = df["ESTABLECIMIENTO"].map(norm_establecimiento)
df["_EST_NORM"] = df["ESTABLECIMIENTO"]  # útil para auditorías de duplicados

# DIRECCION: estandariza abreviaturas recurrentes; corrige “ZONA O”
df["DIRECCION"] = df["DIRECCION"].map(norm_direccion)

# TELEFONO: deja dígitos válidos de 8; separa múltiples con '; '
df["TELEFONO"] = df["TELEFONO"].map(norm_telefono_campo)

# SUPERVISOR / DIRECTOR: normaliza nombre (mayúsculas, sin acentos, espacios)
df["SUPERVISOR"] = df["SUPERVISOR"].map(norm_nombre_persona)
df["DIRECTOR"]   = df["DIRECTOR"].map(norm_nombre_persona)

# NIVEL / SECTOR / AREA / STATUS / MODALIDAD / JORNADA / PLAN / DEPARTAMENTAL
df["NIVEL"]        = map_categorias(df["NIVEL"],        MAP_NIVEL)
df["SECTOR"]       = map_categorias(df["SECTOR"],       MAP_SECTOR)
df["AREA"]         = map_categorias(df["AREA"],         MAP_AREA)
df["STATUS"]       = map_categorias(df["STATUS"],       MAP_STATUS)
df["JORNADA"]      = map_categorias(df["JORNADA"],      MAP_JORNADA)
df["MODALIDAD"]    = df["MODALIDAD"].map(lambda s: norm_texto(s, keep="A-Z0-9 ", to_upper=True))
df["PLAN"]         = df["PLAN"].map(lambda s: norm_texto(s, keep="A-Z0-9 ", to_upper=True))
df["DEPARTAMENTAL"]= df["DEPARTAMENTAL"].map(lambda s: norm_texto(s, keep="A-Z0-9 ", to_upper=True))

# ================================================================
# 2) Chequeos clave (sin eliminar información válida)
# ================================================================
# Nota: Un centro puede tener múltiples CODIGO por nivel/plan/jornada → NO se eliminan.
#     Solo eliminamos duplicados EXACTOS de fila (mismo CODIGO y demás campos).
antes = len(df)
df = df.drop_duplicates()  # exactos
despues = len(df)
print(f"✔ Duplicados exactos eliminados: {antes - despues}")

# Reporte de posibles duplicados por nombre normalizado (sospecha de variantes)
grp_dup = (df.groupby("_EST_NORM")
             .agg(n=("CODIGO","size"),
                  codigos=("CODIGO", lambda s: sorted(s.unique())[:10]),
                  ejemplos=("ESTABLECIMIENTO", lambda s: sorted(s.unique())[:3]))
             .reset_index()
             .sort_values("n", ascending=False))
print("\n=== Sospechas de duplicados por nombre (n>1 puede indicar variantes tipográficas o múltiples servicios) ===")
display(grp_dup.head(20))

# Teléfonos no válidos (NaN tras normalización)
tel_na = df["TELEFONO"].isna().sum()
print(f"\nTELÉFONO -> valores no válidos o vacíos tras limpieza: {tel_na}")

# Departamentos fuera de catálogo (si quedan, revísalos manualmente)
fuera_catalogo = sorted(set(df["DEPARTAMENTO"].dropna()) - DEPARTAMENTOS_OFICIALES)
if fuera_catalogo:
    print("\n⚠ Departamentos fuera de catálogo oficial (revisar mapeos):", fuera_catalogo)
else:
    print("\n✔ Todos los DEPARTAMENTO están en el catálogo oficial.")

# ================================================================
# 3) Orden final de columnas y exportación
# ================================================================

df_limpio = df[CAMPOS].copy()

# Guardar CSV y XLSX
df_limpio.to_csv(CSV_OUT, index=False, encoding="utf-8-sig")
with pd.ExcelWriter(XLSX_OUT, engine="openpyxl") as w:
    df_limpio.to_excel(w, index=False, sheet_name="LIMPIO")

print("\n✔ Exportado CSV:", CSV_OUT)
print("✔ Exportado XLSX:", XLSX_OUT)

# ================================================================
# 4) Mini-resumen para documentar en el informe
# ================================================================
resumen = []
for col in CAMPOS:
    s = df_limpio[col]
    resumen.append({
        "columna": col,
        "nulos": int(s.isna().sum()),
        "% nulos": round(100*s.isna().mean(), 2),
        "valores_únicos": int(s.nunique(dropna=True))
    })
res_df = pd.DataFrame(resumen).sort_values("% nulos", ascending=False)
print("\n=== Resumen post-limpieza ===")
display(res_df)


✔ Duplicados exactos eliminados: 21846

=== Sospechas de duplicados por nombre (n>1 puede indicar variantes tipográficas o múltiples servicios) ===


,_EST_NORM,n,codigos,ejemplos
3415,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA,493,"[00-07-0007-46, 00-12-0149-46, 00-13-0008-46, ...",[INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA]
67,CENTRO DE EDUCACION EXTRAESCOLAR CEEX,66,"[00-01-0696-46, 00-02-0117-46, 00-07-0139-46, ...",[CENTRO DE EDUCACION EXTRAESCOLAR CEEX]
3033,INSTITUTO DE EDUCACION DIVERSIFICADA POR COOPE...,48,"[01-04-9816-46, 05-01-0184-46, 05-01-0271-46, ...",[INSTITUTO DE EDUCACION DIVERSIFICADA POR COOP...
3149,INSTITUTO DIVERSIFICADO POR COOPERATIVA,35,"[01-12-0004-46, 02-01-0607-46, 02-02-0014-46, ...",[INSTITUTO DIVERSIFICADO POR COOPERATIVA]
3440,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA ...,33,"[07-12-0007-46, 11-02-0015-46, 11-03-0020-46, ...",[INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA...
4984,PROGRAMA NACIONAL DE EDUCACION ALTERNATIVA PRONEA,30,"[00-02-0111-46, 00-09-0016-46, 00-12-0119-46, ...",[PROGRAMA NACIONAL DE EDUCACION ALTERNATIVA PR...
2968,INSTITUTO DE COMPUTACION INFORMATICA,29,"[03-01-0257-46, 03-01-1150-46, 03-01-1151-46, ...",[INSTITUTO DE COMPUTACION INFORMATICA]
2769,ESCUELA NORMAL DE EDUCACION FISICA,27,"[02-01-0665-46, 03-01-1231-46, 03-02-1106-46, ...",[ESCUELA NORMAL DE EDUCACION FISICA]
14,ASOCIACION DE MAESTROS DE EDUCACION RURAL DE G...,26,"[13-01-6790-46, 13-03-0055-46, 13-03-0059-46, ...",[ASOCIACION DE MAESTROS DE EDUCACION RURAL DE ...
4843,LICEO SAN JOSE,23,"[02-03-0102-46, 04-02-0025-46, 04-02-0026-46, ...",[LICEO SAN JOSE]



TELÉFONO -> valores no válidos o vacíos tras limpieza: 1888

⚠ Departamentos fuera de catálogo oficial (revisar mapeos): ['CIUDAD CAPITAL']

✔ Exportado CSV: /content/drive/MyDrive/DATA_LIMPIEZA/DATA_COMPLETA_LIMPIA.csv
✔ Exportado XLSX: /content/drive/MyDrive/DATA_LIMPIEZA/DATA_COMPLETA_LIMPIA.xlsx

=== Resumen post-limpieza ===


,columna,nulos,% nulos,valores_únicos
8,DIRECTOR,1906,15.90,5069
6,TELEFONO,1888,15.75,5921
7,SUPERVISOR,533,4.45,686
1,DISTRITO,527,4.40,738
5,DIRECCION,78,0.65,6937
4,ESTABLECIMIENTO,4,0.03,5002
0,CODIGO,0,0.00,11277
3,MUNICIPIO,0,0.00,350
2,DEPARTAMENTO,0,0.00,23
9,NIVEL,0,0.00,1


In [ ]:
# ================================================================
# ➕ EXPORTES ADICIONALES Y RESUMEN (PARTE 4)

# ================================================================
import os
import pandas as pd

# Rutas para reportes
CSV_REP_DUP   = os.path.join(CARPETA, "reporte_sospechas_duplicados_establecimiento.csv")
CSV_REP_RES   = os.path.join(CARPETA, "reporte_resumen_post_limpieza.csv")
CSV_TEL_NA    = os.path.join(CARPETA, "reporte_telefonos_invalidos.csv")
CSV_VCATS     = os.path.join(CARPETA, "reporte_value_counts_categoricos.csv")
CSV_PARES_RAR = os.path.join(CARPETA, "reporte_pares_departamento_municipio_raros.csv")
XLSX_REP      = os.path.join(CARPETA, "reportes_limpieza.xlsx")
MD_RESUMEN    = os.path.join(CARPETA, "resumen_limpieza.md")

# -------- 1) Reporte: Sospechas de duplicados por ESTABLECIMIENTO (n>1) --------
rep_dup = grp_dup[grp_dup["n"] > 1].copy()
rep_dup.to_csv(CSV_REP_DUP, index=False, encoding="utf-8-sig")

# -------- 2) Reporte: Resumen post-limpieza (nulos, únicos) --------
res_df.to_csv(CSV_REP_RES, index=False, encoding="utf-8-sig")

# -------- 3) Teléfonos inválidos (NaN tras normalización) --------
telefonos_invalidos = df[df["TELEFONO"].isna()][["CODIGO","ESTABLECIMIENTO","DEPARTAMENTO","MUNICIPIO","TELEFONO"]].copy()
telefonos_invalidos.to_csv(CSV_TEL_NA, index=False, encoding="utf-8-sig")

# -------- 4) Value counts de variables categóricas --------
cat_cols = ['NIVEL','SECTOR','AREA','STATUS','MODALIDAD','JORNADA','PLAN','DISTRITO','DEPARTAMENTAL']
rows_vc = []
for c in cat_cols:
    if c in df_limpio.columns:
        vc = df_limpio[c].value_counts(dropna=False)
        tmp = vc.reset_index()
        tmp.columns = [c, "conteo"]
        tmp["columna"] = c
        rows_vc.append(tmp)
if rows_vc:
    vcats = pd.concat(rows_vc, ignore_index=True)
else:
    vcats = pd.DataFrame(columns=["columna","valor","conteo"])
vcats.to_csv(CSV_VCATS, index=False, encoding="utf-8-sig")

# -------- 5) Pares Dpto–Muni raros (conteo <= 2) --------
pares = (df_limpio.groupby(["DEPARTAMENTO","MUNICIPIO"]).size()
          .reset_index(name="conteo")
          .sort_values("conteo"))
pares_raros = pares[pares["conteo"] <= 2].copy()
pares_raros.to_csv(CSV_PARES_RAR, index=False, encoding="utf-8-sig")

# -------- 6) Consolidado en un XLSX con varias hojas --------
with pd.ExcelWriter(XLSX_REP, engine="openpyxl") as w:
    rep_dup.to_excel(w, index=False, sheet_name="sospechas_duplicados")
    res_df.to_excel(w, index=False, sheet_name="resumen_post_limpieza")
    telefonos_invalidos.to_excel(w, index=False, sheet_name="telefonos_invalidos")
    vcats.to_excel(w, index=False, sheet_name="value_counts")
    pares_raros.to_excel(w, index=False, sheet_name="pares_dpto_muni_raros")

print("✔ CSV duplicados:", CSV_REP_DUP)
print("✔ CSV resumen post-limpieza:", CSV_REP_RES)
print("✔ CSV teléfonos inválidos:", CSV_TEL_NA)
print("✔ CSV value counts categóricos:", CSV_VCATS)
print("✔ CSV pares Dpto–Muni raros:", CSV_PARES_RAR)
print("✔ XLSX consolidado:", XLSX_REP)

# ================================================================
# 7) RESUMEN EN MARKDOWN: qué se hizo y por qué (con métricas reales)
# ================================================================
n_filas = len(df_limpio)
n_cols = df_limpio.shape[1]
n_dup_exactos = antes - despues  # variables definidas en el pipeline previo
n_tel_na = int(df["TELEFONO"].isna().sum())

# Departamentos fuera de catálogo
try:
    fuera_catalogo_list = sorted(set(df["DEPARTAMENTO"].dropna()) - DEPARTAMENTOS_OFICIALES)
except Exception:
    fuera_catalogo_list = []

resumen_md = f"""# Resumen de Limpieza (Parte 4)

Este proceso implementa las operaciones de limpieza solicitadas en el proyecto para dejar el conjunto **listo para análisis**, manteniendo la trazabilidad y sin eliminar información válida (especialmente considerando que **un centro educativo puede tener más de un código** por **nivel, plan o jornada**).

## Decisiones clave y justificación
- **CODIGO**: se conserva como texto y se eliminan **duplicados exactos** de fila (mismo contenido en todas las columnas) para evitar redundancias. **No** se fusionan ni eliminan filas con **códigos distintos**, ya que un mismo centro puede tener varios servicios autorizados (nivel/plan/jornada).
- **ESTABLECIMIENTO**: normalización (mayúsculas, sin acentos, homogeneización de abreviaturas) para **detectar variaciones tipográficas**. Se reportan **sospechas** de duplicados por nombre normalizado, pero **no** se eliminan automáticamente si difieren en código/servicio.
- **DIRECCION**: se homogeneizan abreviaturas (p. ej., `AV.`→`AVENIDA`, `CALZ.`→`CALZADA`) y se corrige el error frecuente **“ZONA O”→“ZONA 0”**. Se eliminan caracteres no informativos.
- **TELEFONO**: se dejan solo **dígitos**; se validan **8 dígitos** (formato Guatemala); múltiples teléfonos se separan con `; `. Los inválidos quedan como `NaN` para tratamiento posterior.
- **DEPARTAMENTO/MUNICIPIO**: mayúsculas y sin acentos. `DEPARTAMENTO` se contrasta con el **catálogo oficial**; se corrigen equivalencias comunes.
- **Categóricas** (`NIVEL`, `SECTOR`, `AREA`, `STATUS`, `MODALIDAD`, `JORNADA`, `PLAN`, `DISTRITO`, `DEPARTAMENTAL`): homogeneización de valores (p. ej., `MAT.`→`MATUTINA`), conservando la semántica institucional.

## Métricas del resultado
- Filas finales: **{n_filas}**
- Columnas: **{n_cols}**
- Duplicados exactos eliminados: **{n_dup_exactos}**
- Teléfonos no válidos o vacíos (tras limpieza): **{n_tel_na}**

## Chequeos y reportes generados
- **Sospechas de duplicados** por nombre normalizado (`n>1`): `sospechas_duplicados` (CSV y hoja en Excel).
- **Resumen post-limpieza** (nulos, únicos por columna): `resumen_post_limpieza` (CSV y hoja en Excel).
- **Teléfonos inválidos** (NaN tras limpieza): `telefonos_invalidos` (CSV y hoja en Excel).
- **Value counts** de variables categóricas: `value_counts` (CSV y hoja en Excel).
- **Pares raros** DEPARTAMENTO–MUNICIPIO (conteo ≤ 2): `pares_dpto_muni_raros` (CSV y hoja en Excel).

## Notas
- Departamentos fuera de catálogo oficial detectados: {(", ".join(fuera_catalogo_list) if fuera_catalogo_list else "Ninguno")}
- Los archivos limpios finales se guardaron en:
  - **CSV**: `DATA_COMPLETA_LIMPIA.csv`
  - **XLSX**: `DATA_COMPLETA_LIMPIA.xlsx`
"""

with open(MD_RESUMEN, "w", encoding="utf-8") as f:
    f.write(resumen_md)

print("✔ Resumen Markdown:", MD_RESUMEN)


✔ CSV duplicados: /content/drive/MyDrive/DATA_LIMPIEZA/reporte_sospechas_duplicados_establecimiento.csv
✔ CSV resumen post-limpieza: /content/drive/MyDrive/DATA_LIMPIEZA/reporte_resumen_post_limpieza.csv
✔ CSV teléfonos inválidos: /content/drive/MyDrive/DATA_LIMPIEZA/reporte_telefonos_invalidos.csv
✔ CSV value counts categóricos: /content/drive/MyDrive/DATA_LIMPIEZA/reporte_value_counts_categoricos.csv
✔ CSV pares Dpto–Muni raros: /content/drive/MyDrive/DATA_LIMPIEZA/reporte_pares_departamento_municipio_raros.csv
✔ XLSX consolidado: /content/drive/MyDrive/DATA_LIMPIEZA/reportes_limpieza.xlsx
✔ Resumen Markdown: /content/drive/MyDrive/DATA_LIMPIEZA/resumen_limpieza.md


In [ ]:
import pandas as pd

# Cargar data original y limpia
path_original = "/content/drive/MyDrive/DATA_LIMPIEZA/DATA_COMPLETA.csv"   # ajusta a tu ruta real
path_limpia   = "/content/drive/MyDrive/DATA_LIMPIEZA/DATA_COMPLETA_LIMPIA.csv"

df_raw = pd.read_csv(path_original, dtype=str, keep_default_na=False, na_values=[""])
df_limpio = pd.read_csv(path_limpia, dtype=str)

# --- 1) Dimensiones ---
print("=== Dimensiones ===")
print("Original:", df_raw.shape)
print("Limpia  :", df_limpio.shape)

# --- 2) Comparación de nulos ---
nulos_raw = df_raw.isna().sum().to_frame("nulos_original")
nulos_limp = df_limpio.isna().sum().to_frame("nulos_limpia")
comparacion_nulos = nulos_raw.join(nulos_limp)
print("\n=== Comparación de nulos ===")
display(comparacion_nulos)

# --- 3) Ejemplos de cambios de texto ---
print("\n=== Ejemplo cambios: ESTABLECIMIENTO ===")
print("Antes:", df_raw['ESTABLECIMIENTO'].dropna().unique()[:10])
print("Después:", df_limpio['ESTABLECIMIENTO'].dropna().unique()[:10])

print("\n=== Ejemplo cambios: DIRECCION ===")
print("Antes:", df_raw['DIRECCION'].dropna().unique()[:10])
print("Después:", df_limpio['DIRECCION'].dropna().unique()[:10])

print("\n=== Ejemplo cambios: TELEFONO ===")
print("Antes:", df_raw['TELEFONO'].dropna().unique()[:10])
print("Después:", df_limpio['TELEFONO'].dropna().unique()[:10])

# --- 4) Comparación de categorías ---
cat_cols = ["NIVEL","SECTOR","AREA","STATUS","JORNADA"]
for col in cat_cols:
    print(f"\n=== {col} ===")
    vc_raw = df_raw[col].value_counts(dropna=False).rename("original")
    vc_limp = df_limpio[col].value_counts(dropna=False).rename("limpia")
    comparacion = vc_raw.to_frame().join(vc_limp, how="outer").fillna(0).astype(int)
    display(comparacion)


=== Dimensiones ===
Original: (33831, 17)
Limpia  : (11985, 17)

=== Comparación de nulos ===


,nulos_original,nulos_limpia
CODIGO,0,0
DISTRITO,1575,527
DEPARTAMENTO,0,0
MUNICIPIO,0,0
ESTABLECIMIENTO,12,4
DIRECCION,231,78
TELEFONO,2805,1888
SUPERVISOR,1584,533
DIRECTOR,4497,1906
NIVEL,0,0



=== Ejemplo cambios: ESTABLECIMIENTO ===
Antes: ['COLEGIO TECNICO PROGRESISTA CETECPRO'
 "INSTITUTO DE EDUCACION DIVERSIFICADA 'CENTRO DE ESTUDIOS MERCADOLOGICOS Y PUBLICITARIOS'"
 'INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFICADA CENTRO EDUCACIONAL GUATEMALTECO'
 "INSTITUTO DE EDUCACION DIVERSIFICADA 'LICEO MARIANO GALVEZ'"
 'INSTITUTO NORMAL PARA SEÑORITAS CENTRO AMERICA'
 'ESCUELA NACIONAL CENTRAL DE FORMACION SECRETARIAL'
 'ESCUELA NORMAL PARA MAESTROS DE EDUCACION MUSICAL JESUS MARIA ALVARADO'
 'INSTITUTO NACIONAL DE BACHILLERATO EN COMPUTACION'
 'ESCUELA NORMAL DE MAESTROS DE EDUCACION FISICA'
 'ESCUELA NACIONAL CENTRAL DE CIENCIAS COMERCIALES']
Después: ['COLEGIO TECNICO PROGRESISTA CETECPRO'
 'INSTITUTO DE EDUCACION DIVERSIFICADA CENTRO DE ESTUDIOS MERCADOLOGICOS Y PUBLICITARIOS'
 'INSTITUTO PRIVADO MIXTO DE EDUCACION DIVERSIFICADA CENTRO EDUCACIONAL GUATEMALTECO'
 'INSTITUTO DE EDUCACION DIVERSIFICADA LICEO MARIANO GALVEZ'
 'INSTITUTO NORMAL PARA SENORITAS CENTRO AMERICA'
 

,original,limpia
NIVEL,,
DIVERSIFICADO,33831,11985



=== SECTOR ===


...

,original,limpia
AREA,,
RURAL,6168,2198
SIN ESPECIFICAR,9,3
URBANA,27654,9784



=== STATUS ===


,original,limpia
STATUS,,
ABIERTA,19752,7101
CERRADA DEFINITIVAMENTE,5268,1837
CERRADA TEMPORALMENTE,8475,2935
TEMPORAL NOMBRAMIENTO,9,3
TEMPORAL TITULOS,327,109



=== JORNADA ===


,original,limpia
JORNADA,,
DOBLE,11112,3976
INTERMEDIA,384,130
MATUTINA,8811,3087
NOCTURNA,1206,424
SIN JORNADA,3129,1116
VESPERTINA,9189,3252
